In [ ]:
import osmnx as ox
import momepy
import geopandas as gpd
import shapely
from shapely.geometry import LineString
from shapely.ops import unary_union
import os
from dotenv import load_dotenv
import planetary_computer
import pystac_client
import dask.dataframe
import dask_geopandas
import dask.distributed
import deltalake
import shapely.geometry
import contextily
import mercantile
from shapely.geometry import Polygon

In [ ]:
# Define the bounding box coordinates
bbox = [37.935104,48.547183,38.067799,48.633703]

# Create a Polygon from the bounding box
polygon = Polygon([(bbox[0], bbox[1]), 
                   (bbox[0], bbox[3]), 
                   (bbox[2], bbox[3]), 
                   (bbox[2], bbox[1])])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame([1], geometry=[polygon], crs='EPSG:4326')

In [ ]:
area_of_interest = gdf.geometry[0]

In [ ]:
area_of_interest

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)
collection = catalog.get_collection("ms-buildings")

In [ ]:
asset = collection.assets["delta"]

In [ ]:
storage_options = {
    "account_name": asset.extra_fields["table:storage_options"]["account_name"],
    "sas_token": asset.extra_fields["table:storage_options"]["credential"],
}
table = deltalake.DeltaTable(asset.href, storage_options=storage_options)

In [ ]:
quadkeys = [
    int(mercantile.quadkey(tile))
    for tile in mercantile.tiles(*area_of_interest.bounds, zooms=9)
]
quadkeys

In [ ]:
uris = table.file_uris([("quadkey", "in", quadkeys)])
uris

In [ ]:
df = dask_geopandas.read_parquet(uris, storage_options=storage_options)
df

In [ ]:
subset = df.clip(area_of_interest).compute()

In [ ]:
subset.plot()

In [ ]:
buildings = ox.geometries.geometries_from_polygon(area_of_interest, tags={'building':True})

In [ ]:
buildings.plot()